<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-1000)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    67
L    63
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    105
1     62
Name: StarStock, dtype: int64

In [7]:
# v80 stocks
mypf['v80'].value_counts()

0    98
1    69
Name: v80, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1    85
0    82
Name: LatestQtr, dtype: int64

In [9]:
# star + v80 stocks
mypf[mypf['v80'] == 1]['StarStock'].value_counts()

0    44
1    25
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    147
1     20
Name: InPortfolio, dtype: int64

In [11]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [12]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [13]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['v80'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['v80'].values[0] 
    df_prec_dev = df_prec_dev.append(tmp)

In [14]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-17 00:00:00+05:30,17707.0,^NSEI,0.79,0.56,-0.07,18812.0,10806.0,13.80,6.0
2023-04-24 00:00:00+05:30,17743.0,^NSEI,0.73,0.59,-0.03,18812.0,10806.0,13.35,6.0
2023-05-02 00:00:00+05:30,18148.0,^NSEI,2.73,0.55,0.07,18812.0,10806.0,8.29,4.0
2023-05-09 00:00:00+05:30,18266.0,^NSEI,3.09,0.59,0.13,18812.0,10806.0,6.82,3.0
2023-05-16 00:00:00+05:30,18286.0,^NSEI,2.92,0.57,0.44,18812.0,10806.0,6.57,3.0


In [15]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [16]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [17]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'v80']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 20


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
VALIANTORG.NS,481.0,-16.81,-0.87,1.32,93.13,285.0,481.71,24.0,15.1,M,37.0,0,0,0
GLAND.NS,1355.0,-21.33,-3.14,0.54,93.64,219.0,1354.78,45.2,22.5,M,50.0,0,0,0
LUXIND.NS,1494.0,-6.03,-1.26,1.14,89.32,204.0,1499.87,21.4,24.9,M,-16.0,0,0,0
VAIBHAVGBL.NS,320.0,-0.19,0.31,-0.32,92.95,202.0,320.64,35.4,47.5,M,-34.0,0,0,0
INDIGOPNTS.NS,1317.0,0.41,-0.61,2.54,84.35,136.0,1215.89,104.9,53.3,M,49.0,0,1,0
JUBLFOOD.NS,467.0,-11.10,-0.95,0.67,84.50,93.0,458.16,90.5,73.2,H,19.0,0,1,0
IRCTC.NS,626.0,-5.27,0.15,0.33,59.32,86.0,623.94,77.4,53.1,H,31.0,0,1,0
HDFCAMC.NS,1842.0,-6.43,-0.25,0.17,88.68,75.0,1856.05,38.5,27.6,M,28.0,1,0,1
DMART.NS,3479.0,-11.06,-0.41,0.32,55.09,53.0,3600.56,160.1,95.0,H,41.0,1,1,0


In [18]:
# top 7 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
VALIANTORG.NS,481.0,-16.81,-0.87,1.32,93.13,285.0,481.71,24.0,15.1,M,37.0,0,0,0
GLAND.NS,1355.0,-21.33,-3.14,0.54,93.64,219.0,1354.78,45.2,22.5,M,50.0,0,0,0
LUXIND.NS,1494.0,-6.03,-1.26,1.14,89.32,204.0,1499.87,21.4,24.9,M,-16.0,0,0,0
VAIBHAVGBL.NS,320.0,-0.19,0.31,-0.32,92.95,202.0,320.64,35.4,47.5,M,-34.0,0,0,0
INDIGOPNTS.NS,1317.0,0.41,-0.61,2.54,84.35,136.0,1215.89,104.9,53.3,M,49.0,0,1,0
JUBLFOOD.NS,467.0,-11.10,-0.95,0.67,84.50,93.0,458.16,90.5,73.2,H,19.0,0,1,0
IRCTC.NS,626.0,-5.27,0.15,0.33,59.32,86.0,623.94,77.4,53.1,H,31.0,0,1,0


In [19]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [20]:
# star + v80 stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] == 1)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ICICIGI.NS,1117.0,-4.02,-0.52,0.09,1620.0,1060.0,89.82,45.0,1.0,0.0,M,1,1,1
BERGEPAINT.NS,612.0,1.17,0.50,1.01,858.0,533.0,75.69,40.0,1.0,0.0,M,1,1,1
OFSS.NS,3550.0,19.64,0.81,1.77,4392.0,2442.0,43.18,24.0,1.0,0.0,M,1,1,1
UJJIVANSFB.NS,34.0,30.77,3.85,3.70,42.0,13.0,27.59,24.0,0.0,1.0,M,1,1,1
DABUR.NS,531.0,-3.85,-0.36,-0.19,645.0,464.0,62.98,21.0,1.0,0.0,H,1,1,1


In [21]:
# top 5 from star + v80 for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ICICIGI.NS,1117.0,-4.02,-0.52,0.09,1620.0,1060.0,89.82,45.0,1.0,0.0,M,1,1,1
DABUR.NS,531.0,-3.85,-0.36,-0.19,645.0,464.0,62.98,21.0,1.0,0.0,H,1,1,1
BERGEPAINT.NS,612.0,1.17,0.50,1.01,858.0,533.0,75.69,40.0,1.0,0.0,M,1,1,1
OFSS.NS,3550.0,19.64,0.81,1.77,4392.0,2442.0,43.18,24.0,1.0,0.0,M,1,1,1
UJJIVANSFB.NS,34.0,30.77,3.85,3.70,42.0,13.0,27.59,24.0,0.0,1.0,M,1,1,1


In [22]:
# star + v80 stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] == 1)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
HDFCBANK.NS,1647.0,6.35,0.97,0.37,1708.0,1003.0,8.65,4.0,1.0,0.0,H,1,1,1
EQUITASBNK.NS,78.0,37.37,3.51,1.43,80.0,33.0,4.26,3.0,1.0,1.0,M,1,1,1
HINDUNILVR.NS,2676.0,4.24,0.08,0.48,2728.0,1897.0,6.26,2.0,1.0,1.0,H,1,1,1
ITC.NS,424.0,21.48,1.72,1.01,431.0,146.0,2.46,2.0,1.0,1.0,H,0,1,1
BRITANNIA.NS,4607.0,12.83,1.13,0.55,4635.0,2945.0,1.66,1.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,946.0,7.30,1.02,1.13,953.0,333.0,1.13,1.0,1.0,1.0,H,1,1,1
CERA.NS,7439.0,32.18,2.33,0.99,7439.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,1
POLYCAB.NS,3426.0,24.50,2.00,0.95,3426.0,793.0,0.00,0.0,1.0,0.0,M,1,1,1
TITAN.NS,2790.0,9.59,1.10,1.37,2790.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1


In [23]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] != 1)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ADANIGREEN.NS,874.0,-47.37,-3.61,4.48,2970.0,371.0,80.65,240.0,-1.0,-1.0,L,1,1,0
ADANIENT.NS,1889.0,-34.90,-0.86,3.01,4165.0,233.0,57.88,120.0,1.0,0.0,M,1,1,0
BANDHANBNK.NS,244.0,-0.29,-1.22,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0
LTIM.NS,4736.0,3.38,0.55,-0.35,7439.0,2267.0,52.26,57.0,1.0,0.0,M,1,1,0
GODREJIND.NS,469.0,6.32,0.23,1.16,644.0,354.0,60.34,37.0,0.0,1.0,L,0,1,0
TATAPOWER.NS,211.0,-1.70,-0.47,0.00,287.0,49.0,31.93,36.0,1.0,0.0,M,1,1,0
BAJAJFINSV.NS,1428.0,-5.53,0.66,0.38,1907.0,542.0,35.09,34.0,0.0,0.0,M,1,1,0
SBICARD.NS,878.0,7.14,0.00,1.70,1133.0,673.0,55.43,29.0,1.0,0.0,M,1,1,0
CIPLA.NS,922.0,-10.30,-0.10,-0.33,1173.0,701.0,53.18,27.0,1.0,0.0,M,1,1,0


In [24]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
AUBANK.NS,729.0,15.70,1.11,1.75,729.0,317.0,0.00,0.0,1.0,0.0,M,1,1,0
CERA.NS,7439.0,32.18,2.33,0.99,7439.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,1
POLYCAB.NS,3426.0,24.50,2.00,0.95,3426.0,793.0,0.00,0.0,1.0,0.0,M,1,1,1
TITAN.NS,2790.0,9.59,1.10,1.37,2790.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1
BRITANNIA.NS,4607.0,12.83,1.13,0.55,4635.0,2945.0,1.66,1.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,946.0,7.30,1.02,1.13,953.0,333.0,1.13,1.0,1.0,1.0,H,1,1,1
HINDUNILVR.NS,2676.0,4.24,0.08,0.48,2728.0,1897.0,6.26,2.0,1.0,1.0,H,1,1,1
ITC.NS,424.0,21.48,1.72,1.01,431.0,146.0,2.46,2.0,1.0,1.0,H,0,1,1
EQUITASBNK.NS,78.0,37.37,3.51,1.43,80.0,33.0,4.26,3.0,1.0,1.0,M,1,1,1


In [25]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ACC.NS,1784.0,-18.23,-0.87,0.23,2748.0,1259.0,64.74,54.0,0.0,0.0,L,0,0,0
PGHH.NS,13847.0,-0.67,0.01,0.14,15778.0,9190.0,29.31,14.0,0.0,0.0,L,0,0,1
PVRINOX.NS,1435.0,-1.23,NaN,-0.34,1483.0,1435.0,100.00,3.0,1.0,-1.0,L,0,0,0
NESTLEIND.NS,21780.0,11.72,0.93,1.65,22050.0,14623.0,3.64,1.0,1.0,1.0,H,0,0,1
SIEMENS.NS,3872.0,28.60,1.73,1.68,3891.0,1143.0,0.69,0.0,1.0,1.0,H,0,0,0


In [26]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
INDUSTOWER.NS,155.0,-14.11,-2.22,-0.67,303.0,137.0,89.16,95.0,1.0,1.0,H,1,0,0
BANDHANBNK.NS,244.0,-0.29,-1.22,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0


In [27]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
INDUSTOWER.NS,155.0,-14.11,-2.22,-0.67,303.0,137.0,89.16,95.0,1.0,1.0,H,1,0,0
BANDHANBNK.NS,244.0,-0.29,-1.22,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0
HDFCLIFE.NS,560.0,3.04,0.18,1.36,757.0,461.0,66.55,35.0,1.0,1.0,H,1,0,0


In [28]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 15


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
CERA.NS,7439.0,32.18,2.33,0.99,7439.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,1
TITAN.NS,2790.0,9.59,1.10,1.37,2790.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1
SIEMENS.NS,3872.0,28.60,1.73,1.68,3891.0,1143.0,0.69,0.0,1.0,1.0,H,0,0,0
ICICIBANK.NS,946.0,7.30,1.02,1.13,953.0,333.0,1.13,1.0,1.0,1.0,H,1,1,1
BRITANNIA.NS,4607.0,12.83,1.13,0.55,4635.0,2945.0,1.66,1.0,1.0,1.0,H,1,1,1
ITC.NS,424.0,21.48,1.72,1.01,431.0,146.0,2.46,2.0,1.0,1.0,H,0,1,1
NESTLEIND.NS,21780.0,11.72,0.93,1.65,22050.0,14623.0,3.64,1.0,1.0,1.0,H,0,0,1
EQUITASBNK.NS,78.0,37.37,3.51,1.43,80.0,33.0,4.26,3.0,1.0,1.0,M,1,1,1
BANKBARODA.NS,186.0,18.04,2.53,1.16,193.0,39.0,4.55,4.0,1.0,1.0,H,1,1,0


In [29]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
NESTLEIND.NS,21780.0,11.72,0.93,1.65,22050.0,14623.0,3.64,1.0,1.0,1.0,H,0,0,1
TITAN.NS,2790.0,9.59,1.10,1.37,2790.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1
HDFCLIFE.NS,560.0,3.04,0.18,1.36,757.0,461.0,66.55,35.0,1.0,1.0,H,1,0,0
COLPAL.NS,1661.0,7.09,0.19,1.17,1741.0,1262.0,16.70,5.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,946.0,7.30,1.02,1.13,953.0,333.0,1.13,1.0,1.0,1.0,H,1,1,1
HAVELLS.NS,1291.0,4.89,0.16,0.82,1461.0,606.0,19.88,13.0,1.0,1.0,H,1,0,1
MARUTI.NS,9071.0,3.70,0.30,0.63,9528.0,6217.0,13.80,5.0,1.0,1.0,H,1,0,0
HINDUNILVR.NS,2676.0,4.24,0.08,0.48,2728.0,1897.0,6.26,2.0,1.0,1.0,H,1,1,1


In [30]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
BANDHANBNK.NS,244.0,-0.29,-1.22,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0
INDUSTOWER.NS,155.0,-14.11,-2.22,-0.67,303.0,137.0,89.16,95.0,1.0,1.0,H,1,0,0


In [31]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 51


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ADANITRANS.NS,812.0,-66.16,-4.04,1.26,4105.0,235.0,85.09,406.0,1.0,-1.0,L,0,0,0
PEL.NS,748.0,-21.28,-5.05,-0.98,2866.0,635.0,94.94,283.0,-1.0,0.0,L,1,0,0
ADANIGREEN.NS,874.0,-47.37,-3.61,4.48,2970.0,371.0,80.65,240.0,-1.0,-1.0,L,1,1,0
IBREALEST.NS,57.0,-20.97,0.00,0.00,191.0,47.0,93.06,235.0,-1.0,-1.0,L,0,0,0
JCHAC.NS,1099.0,-11.64,-1.85,-1.16,2837.0,1017.0,95.49,158.0,NaN,NaN,M,0,0,1
TEAMLEASE.NS,2256.0,-15.91,-2.65,-1.37,5281.0,2028.0,92.99,134.0,NaN,NaN,M,0,0,1
ADANIENT.NS,1889.0,-34.90,-0.86,3.01,4165.0,233.0,57.88,120.0,1.0,0.0,M,1,1,0
LALPATHLAB.NS,2033.0,-8.53,-0.18,0.32,4186.0,1748.0,88.31,106.0,NaN,NaN,M,1,0,1
DIXON.NS,2913.0,-20.51,-1.23,0.62,5750.0,1596.0,68.30,97.0,NaN,NaN,M,0,0,1


In [32]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
PVRINOX.NS,1435.0,-1.23,NaN,-0.34,1483.0,1435.0,100.0,3.0,1.0,-1.0,L,0,0,0


In [33]:
# save the output
#df_prec_dev = df_prec_dev.reset_index()
#df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/Nifty100-Upside-Analysis-ATH.csv', index=False)

In [34]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('VALIANTORG.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-17 00:00:00+05:30,505.0,VALIANTORG.NS,-13.68,-0.85,-0.47,1851.0,380.0,91.50,267.0
2023-04-24 00:00:00+05:30,516.0,VALIANTORG.NS,-11.58,-0.34,1.38,1851.0,380.0,90.75,259.0
2023-05-02 00:00:00+05:30,516.0,VALIANTORG.NS,-11.45,-0.34,1.36,1851.0,380.0,90.75,259.0
2023-05-09 00:00:00+05:30,500.0,VALIANTORG.NS,-14.00,-0.34,2.00,1851.0,380.0,91.84,270.0
2023-05-16 00:00:00+05:30,481.0,VALIANTORG.NS,-16.81,-0.87,1.32,1851.0,380.0,93.13,285.0
